# Configuring an Experiment with UW Tools

Given multiple configuration files like the ones below, `uwtools` allows us to combine them into one comprehensive configuration file for an experiment. Suppose we start with a base configuration file like the one below, which is a truncated version of the default config for SRW, using FV3GFS initial conditions (ICs) and the FV3_GFS_v16 physics suite. 

<div class="alert alert-warning"><b>Note: </b>This notebook was tested using <code>uwtools</code> version 2.4.2.</div>
<!--cell 0-->

In [1]:
%%bash
cat fixtures/config-exp/base-file.yaml

user:
  ACCOUNT: ""
  MACHINE: ""
workflow:
  EXPTDIR: ""
task_make_ics:
  chgres_cube:
    namelist:
      update_values:
        config:
          cycle_day: !int "{{ cycle.strftime('%d') }}"
          cycle_hour: !int "{{ cycle.strftime('%H') }}"
          cycle_mon: !int "{{ cycle.strftime('%m') }}"
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: "{{ task_make_ics.chgres_cube.rundir }}"
          external_model: "FV3GFS"
          input_type: "gaussian_nemsio"
          tg3_from_soil: false
          tracers:
            - sphum
            - liq_wat
          tracers_input:
            - spfh
            - clwmr
          varmap_file: "{{ user.PARMdir }}/ufs_utils/varmap_tables/GFSphys_var_map.txt"
    rundir: '{{ workflow.EXPTDIR }}/make_ics'


If, instead, HRRR ICs are needed for the experiment we want to run, we can apply changes from a config that describes the set of changes related to the initial condition source.
<!--cell 2-->

In [2]:
%%bash
cat fixtures/config-exp/hrrr-ics.yaml

task_make_ics:
  chgres_cube:
    namelist:
      update_values:
        config:
          convert_nst: False
          external_model: "HRRR"
          geofrid_file_input_grid: "{{ workflow.FIXgsm }}/geo_em.d01.nc_HRRRX"
          input_type: "grib2"
          nsoil_out: 9
          tg3_from_soil: True
          tracers: ""
          tracers_input: ""


A different physics suite, such as the FV3_RAP physics suite may also be used.
<!--cell 4-->

In [3]:
%%bash
cat fixtures/config-exp/fv3-rap-physics.yaml

task_make_ics:
  chgres_cube:
    namelist:
      update_values:
        config:
          varmap_file: "{{ user.PARMdir }}/ufs_utils/varmap_tables/GSDphys_var_map.txt"


There will be some information each user will need to provide. Those can be included in a user config file. Separating these items into their own config file will help guide an easier user experience. 
<!--cell 6-->

In [4]:
%%bash
cat fixtures/config-exp/user.yaml

user:
  ACCOUNT: zrtrr
  MACHINE: hera
  PARMdir: /path/to/ufs-srweather-app/parm
workflow: 
  EXPTDIR: /path/to/my/output


Here, the `configure_experiment()` function takes a list of config file paths and combines them into a comprehensive config object. The `get_yaml_config()` function initially creates an empty `YAMLConfig` object which is then built up using the list of config files and `update_from()`. After all of the key-value pairs are coalesced into one object, `dereference()` renders any Jinja 2 expressions wherever possible.
<!--cell 8-->

In [5]:
from uwtools.api.config import get_yaml_config

def configure_experiment(cfg_files: list[str]):
    base_cfg = get_yaml_config({})
    for cfg_file in cfg_files:
        cfg = get_yaml_config(cfg_file)
        base_cfg.update_from(cfg)
    base_cfg.dereference()
    return base_cfg

Here the list of config file paths is created and passed to `configure_experiment()`. Note that if the configs share any keys, the values from files later in the list will override the earlier ones. For example, `convert_nst:` had a `true` value in the base config file, but it was changed to `false` by the HRRR ICs config. While some Jinja 2 expressions were not able to be rendered and still exist here, the keys `data_dir_input_grid:` and `rundir:` have had their Jinja 2 expressions rendered. 
<!--cell 10-->

In [6]:
user_config_files = ['fixtures/config-exp/base-file.yaml',
                     'fixtures/config-exp/hrrr-ics.yaml',
                     'fixtures/config-exp/fv3-rap-physics.yaml',
                     'fixtures/config-exp/user.yaml'
                    ]
configure_experiment(user_config_files)

user:
  ACCOUNT: zrtrr
  MACHINE: hera
  PARMdir: /path/to/ufs-srweather-app/parm
workflow:
  EXPTDIR: /path/to/my/output
task_make_ics:
  chgres_cube:
    namelist:
      update_values:
        config:
          cycle_day: !int '{{ cycle.strftime(''%d'') }}'
          cycle_hour: !int '{{ cycle.strftime(''%H'') }}'
          cycle_mon: !int '{{ cycle.strftime(''%m'') }}'
          convert_atm: true
          convert_nst: false
          convert_sfc: true
          data_dir_input_grid: /path/to/my/output/make_ics
          external_model: HRRR
          input_type: grib2
          tg3_from_soil: true
          tracers: ''
          tracers_input: ''
          varmap_file: /path/to/ufs-srweather-app/parm/ufs_utils/varmap_tables/GSDphys_var_map.txt
          geofrid_file_input_grid: '{{ workflow.FIXgsm }}/geo_em.d01.nc_HRRRX'
          nsoil_out: 9
    rundir: /path/to/my/output/make_ics